In [2]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten,Input
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l1 ,l2
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [9]:
x_train.shape

(50000, 32, 32, 3)

In [10]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
# Dense Block
def denseblock(input, num_filter = 16, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(wd))(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck


def transition(input, num_filter = 16, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(wd))(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
# Dense Block
def add_denseblock(input, l, num_filter = 16, dropout_rate = 0.2):
  temp = input
  for _ in range(int(l)):
    BatchNorm = BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(temp)
    relu = Activation('relu')(BatchNorm)
    Conv2D_1_1 = Conv2D(int(num_filter*4),(1,1), use_bias= False, padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(wd))(relu)
    if dropout_rate>0:
      Conv2D_1_1 = Dropout(dropout_rate)(Conv2D_1_1)
    BatchNorm = BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(Conv2D_1_1)
    relu = Activation('relu')(BatchNorm)
    Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(wd))(relu)
    if dropout_rate>0:
      Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
    concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
    temp = concat
        
  return temp


In [ ]:
def add_transition(input,  dropout_rate = 0.2,num_filter = 16,):
    BatchNorm = BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter), (1,1), use_bias=False ,padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(wd))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D()(Conv2D_BottleNeck)
    
    return avg

In [ ]:
def output_layer(input):
    BatchNorm = BatchNormalization(beta_regularizer=l2(wd),gamma_regularizer=l2(wd))(input)
    relu = Activation('relu')(BatchNorm)
    GP = GlobalAveragePooling2D()(relu)
    output = Dense(num_classes, activation='softmax',kernel_regularizer=l2(wd))(GP)
    
    return output

In [ ]:
wd=1e-4
global wd
growth_rate=13
num_filter=4*growth_rate
reduction=0.5
depth=100
reduction_1=0.5
reduction_2=0.5
no_layer=(depth-4)//6
dropout_rate=0.2


input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same', kernel_initializer='he_uniform', bias_initializer='zeros',
                      kernel_regularizer=l2(wd))(input)

First_Block = add_denseblock(First_Conv2D,no_layer ,growth_rate ,dropout_rate)
num_filter=math.floor((num_filter+no_layer*growth_rate))*reduction_1
First_Transition = add_transition(First_Block, dropout_rate, num_filter=num_filter)

Second_Block = add_denseblock(First_Transition,no_layer,growth_rate,dropout_rate)
num_filter=math.floor((num_filter+no_layer*growth_rate))*reduction_2
Second_Transition = add_transition(Second_Block, dropout_rate, num_filter=num_filter)

Third_Block = add_denseblock(Second_Transition,no_layer,growth_rate,dropout_rate )
#Third_Transition = add_transition(Third_Block, dropout_rate, num_filter=171)

#Last_Block = add_denseblock(Third_Transition, dropout_rate,l, num_filter)
output = output_layer(Third_Block)

In [ ]:
wd=1e-4
global wd
num_filter = 52
num_layer=16
dropout_rate = 0.2
l = 12
input =Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same', kernel_initializer='he_uniform', bias_initializer='zeros',
                      kernel_regularizer=l2(wd))(input)

First_Block = denseblock(First_Conv2D, num_layer, dropout_rate)
First_Transition = transition(First_Block, num_layer, dropout_rate)

Second_Block = denseblock(First_Transition, num_layer, dropout_rate)
Second_Transition = transition(Second_Block, num_layer, dropout_rate)

Third_Block = denseblock(Second_Transition, num_layer, dropout_rate)
Third_Transition = transition(Third_Block, num_layer, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_layer, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
#https://arxiv.org/pdf/1608.06993.pdf
from IPython.display import IFrame, YouTubeVideo
YouTubeVideo(id='-W6y8xnd--U', width=600)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 52)   1404        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 52)   208         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 52)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
print(len(model.layers))

262


In [33]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-1,decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [46]:
filepath="/content/sample_data/weights_densenetbc_6.best.hdf5"
#callbacks
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
checkpoint_save = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint_save,learning_rate_reduction]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:

from keras.preprocessing import image
#data augmentation
datagen = image.ImageDataGenerator(
    zoom_range=0.3,
    rotation_range=15,
    horizontal_flip=True,
    fill_mode='nearest',
    
    )
datagen.fit(X_train)

In [ ]:
batch_size=60
epochs=80
#Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, y_train,
                                 batch_size=batch_size),
                        steps_per_epoch=len(X_train) / batch_size,
                        epochs=epochs,validation_data=(X_test, y_test),callbacks=callbacks_list)  

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
833/833 [==============================] - 175s 210ms/step - loss: 1.5529 - accuracy: 0.5887 - val_loss: 1.7967 - val_accuracy: 0.5629

Epoch 00001: val_accuracy improved from -inf to 0.56290, saving model to /content/sample_data/weights_densenetbc_6.best.hdf5
Epoch 2/100
833/833 [==============================] - 182s 218ms/step - loss: 1.5031 - accuracy: 0.6044 - val_loss: 1.5011 - val_accuracy: 0.6236

Epoch 00002: val_accuracy improved from 0.56290 to 0.62360, saving model to /content/sample_data/weights_densenetbc_6.best.hdf5
Epoch 3/100
833/833 [==============================] - 175s 210ms/step - loss: 1.4703 - accuracy: 0.6151 - val_loss: 1.5617 - val_accuracy: 0.6048

Epoch 00003: val_accuracy did not improve from 0.62360
Epoch 4/100
833/833 [==============================] - 175s 210ms/step - loss: 1.4335 - accuracy: 0.6281 - val_loss: 1.5401 - val_accuracy: 0.6254

Epoch 00004: val_accuracy improved from 0.62360 to 0.62540, saving model to /content/sample_data/wei

In [ ]:
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, 
                    validation_data=(X_test, y_test)) 

In [40]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 14s 45ms/step - loss: 1.6812 - accuracy: 0.5648
Test loss: 1.6811939477920532
Test accuracy: 0.5648000240325928


In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")